# Inquiries

In [1]:
# Make some helpful functions 
import os
import sys
import pandas as pd
import numpy as np
import subprocess
import glob
from functools import partial 
from os import chdir
import io
import time
import matplotlib.pyplot as plt
import seaborn as sns

def shell_do(command):
    print(f'Executing: {command}', file=sys.stderr)
    !$command

def submitTerminal(command, printing=False, message=''):
    # quick command to submit jobs to terminal
    start = time.time()
    res=subprocess.run(command.split(' '), stdout=subprocess.PIPE)
    end = time.time()
    sys.stdout.write('EXEC_TIME in sec: '+ str(round(end - start, 3)) + ' : ')
    if printing:
        print(res.stdout.decode('utf-8'))
    if message=='':
        return(res.stdout.decode('utf-8'))
    else:
        print(message, '\n')
        
def plink2_cmd(plink_file_path, output_path, options):
    cmd = f'/data/CARD/PD/AMP-PD/Plink/plink2_dev_June8/plink2 --pfile {plink_file_path} {options} --out {output_path} --threads 2'
    return cmd
        
def plink_cmd(plink_file_path, output_path, options):
    cmd = f'/data/CARD/PD/AMP-PD/Plink/plink2_dev_June8/plink2 --bfile {plink_file_path} {options} --out {output_path} --threads 2'
    return cmd

def plink1_cmd(plink_file_path, output_path, options):
    cmd = f'plink --bfile {plink_file_path} {options} --out {output_path} --threads 2'
    return cmd

## LRRK2 R1441G/C
From Andy 11/29/2021

"do you happen to know if there are any R1441C mutation carriers in PPMI – I’m pretty sure there are R1441G, but don’t know about the C allele"
* R1441G: chr12:40310434:C:G
* R1441C: chr12:40310434:C:T

In [90]:
plink_file_path = '/data/CARD/PD/AMP-PD/Plink/2021_v2_5release/formatted_split_normalized_pfile/reformatted_split_normalized_chr12'

t = submitTerminal(plink2_cmd(f'{plink_file_path}', 'R1441', f'--snps chr12:40310434:C:T,chr12:40310434:C:G --export A'), message='extract snps of interest from the PLINK2 AMP v2.5 files')

EXEC_TIME in sec: 10.199 : extract snps of interest from the PLINK2 AMP v2.5 files 



In [91]:
d = pd.read_csv('R1441.raw', sep='\t')
d.pivot_table(index='chr12:40310434:C:T_C', columns=['chr12:40310434:C:G_C'], values='IID', aggfunc='count')

chr12:40310434:C:G_C,1,2
chr12:40310434:C:T_C,,
1,NaN,11.0
2,91.0,10316.0


In [92]:
d['R1441G'] = 2-d['chr12:40310434:C:G_C']
d['R1441C'] = 2-d['chr12:40310434:C:T_C']
d['DATASET'] = [x[:2] for x in d.IID]
d = d.drop(columns=['PAT', 'MAT', 'SEX', 'PHENOTYPE', 'chr12:40310434:C:G_C', 'chr12:40310434:C:T_C'])

In [93]:
dcov = pd.read_csv('/data/CARD/PD/AMP-PD/release2.5_COVFILE/AMPPD_releasev2.5_covariates_JUNE_2021.csv')
df = pd.merge(d, dcov, left_on='IID', right_on='ID')

In [94]:
d.pivot_table(index='DATASET', columns=['R1441G', 'R1441C'], values='IID', aggfunc='count', fill_value=0)

R1441G      0      1
R1441C      0  1   0
DATASET             
BF        171  1   0
HB       1173  0   0
LB       4579  0   0
LC        542  4  53
PD       1495  4   1
PP       1769  1  37
SU        259  0   0
SY        328  1   0

In [95]:
df['dataset'] = [x if pd.isna(a) else x if 'Genetic' not in a else f'PP-{a}' for x, a in zip(df.DATASET, df.ENROLL_STUDY_ARM)]

In [96]:
df.pivot_table(index='dataset', columns=['R1441G', 'R1441C'], values='IID', aggfunc='count', fill_value=0)

R1441G                             0      1
R1441C                             0  1   0
dataset                                    
BF                               171  1   0
HB                              1173  0   0
LB                              4579  0   0
LC                               542  4  53
PD                              1495  4   1
PP                               811  0   0
PP-Genetic Cohort PD             245  1  16
PP-Genetic Cohort Unaffected     378  0  16
PP-Genetic Registry PD           142  0   2
PP-Genetic Registry Unaffected   193  0   3
SU                               259  0   0
SY                               328  1   0

In [97]:
# df.IID[(df.DATASET=='PP')&(df.R1441C==1)]

## Create Polygenic Risk Scores

From MichaelB and Kalpana
12/12/2021

"Could you send separate scores excluding only the G2019S (I didn’t see 1441C/G in your PRS list) or only the GBA N370S variant? In other words, do not exclude both simultaneously."

In [31]:
# AMP-PD v2.5 plink file reduced to 90 risk associated SNPs
## Please refer to the folloing path for more detail
## '/data//CARD/PD/AMP-PD/Plink/2021_v2_5release/prs_AMPv2.5/PRS_calc_distribution_forAMPv2.5.html'
pfile_path='/data/CARD/PD/AMP-PD/Plink/2021_v2_5release/prs_AMPv2.5/meta5-v3'

# Score file (defining the score per allele)
score_path = '/data/CARD/PD/AMP-PD/Plink/2021_v2_5release/prs_AMPv2.5/META5_GRS_rsID-replaced2SNPs_wSNPsinLD.txt'

In [41]:
sc = pd.read_csv(score_path, sep='\t', header=None)
sc.head()
sc_excl_g2019s=sc[sc[0]!='rs34637584'].copy()
sc_excl_n370s=sc[sc[0]!='rs76763715'].copy()
sc.to_csv('score.txt', header=False, index=False, sep='\t')
sc_excl_g2019s.to_csv('score_noG2019S.txt', header=False, index=False, sep='\t')
sc_excl_n370s.to_csv('score_noN370S.txt', header=False, index=False, sep='\t')

In [51]:
# create prs
plink_file_path = '/data/CARD/PD/AMP-PD/Plink/2021_v2_5release/pfile/amp_v2.5_allchr'
for score_path in ['score.txt', 'score_noG2019S.txt', 'score_noN370S.txt']:
    t = submitTerminal(plink2_cmd(f'{pfile_path}', f'{score_path.replace(".txt","")}',
                                  f'--score {score_path}'), 
                       message='extract all META5 variants from the PLINK2 AMP v2.5 files')

EXEC_TIME in sec: 0.088 : extract all META5 variants from the PLINK2 AMP v2.5 files 

EXEC_TIME in sec: 0.031 : extract all META5 variants from the PLINK2 AMP v2.5 files 

EXEC_TIME in sec: 0.033 : extract all META5 variants from the PLINK2 AMP v2.5 files 



In [69]:
# Join with Global PCs
d1 = pd.read_csv('score.sscore', index_col = 'IID', sep='\t')
d2 = pd.read_csv('/data/CARD/PD/AMP-PD/Plink/2021_v2_5release/euro_king_pca_v2.5_July2021/genetic_ancestry_all_pca.csv', index_col='IID')
d = pd.concat([d1,d2],axis=1)

# They only need PPMI samples
d = d[d.index.str.contains('PP-')].copy()
d.shape

(1807, 16)

In [98]:
d.to_csv('score_pcs.csv')

In [97]:
# chck the "missing" PRS
mis = pd.read_csv('PATNOs Without PRS90 03JAN22.csv')
mis['IID'] = 'PP-' + mis.PATNO.astype('str')
mis = mis.set_index('IID').copy()
mis['dummy']=1
mis.join(d, how='left').fillna(0).pivot_table(index='APPRDX', columns=['InfPop'], values=['dummy'], aggfunc='count')


dummy                         
InfPop     0 ADMIX AFRICA ASIA EUROPE
APPRDX                               
1        7.0   NaN    1.0  1.0    6.0
2        2.0   NaN    NaN  NaN    1.0
3        1.0   NaN    NaN  NaN    1.0
4        1.0   NaN    NaN  NaN    NaN
5       21.0   NaN    NaN  NaN   76.0
6       37.0   1.0    NaN  NaN  168.0

## GBA,LRRK2 status and UPSIT
From Ken 1/3/2022

"I need a listing by subject including Age, Sex, Raw UPSIT score (number correct), Baseline total UPDRS, Genetic variant (or sporadic PD)"

        'rs2236288',  # GBA E326K
        'rs76763715',  # GBA N370N
        'rs34637584',  # LRRK2 G2019S
        'rs104893877',   # SNCA A53T


In [51]:
# UPSIT (in LC, sometimes measured multiple times.)
d1 =  pd.read_csv("/data/CARD/PD/AMP-PD/Clinical/2021_v2_5release/clinical/UPSIT.csv").drop_duplicates()

# Demographics, enrollment category case-control status
d2 = pd.read_csv('/data/CARD/PD/AMP-PD/Clinical/2021_v2_5release/clinical/Demographics.csv').drop_duplicates()
d3 = pd.read_csv('/data/CARD/PD/AMP-PD/Clinical/2021_v2_5release/clinical/Enrollment.csv').drop_duplicates()
d4 = pd.read_csv('/data/CARD/PD/AMP-PD/Clinical/2021_v2_5release/amp_pd_case_control.csv').drop_duplicates()
t = pd.merge(d1[['participant_id','visit_name', 'visit_month', 'upsit_total_score']], d2[['participant_id', 'age_at_baseline', 'sex']], on='participant_id', how='left')
t2 = pd.merge(t, d3[['participant_id', 'study_arm', 'prodromal_category']], on='participant_id',  how='left')
t3 = pd.merge(t2, d4, on='participant_id',  how='left').drop_duplicates()

In [52]:
# age at diagnosis (at visit_month==0)
d5 = pd.read_csv('/data/CARD/PD/AMP-PD/Clinical/2021_v2_5release/clinical/PD_Medical_History.csv').drop_duplicates()
d5 = d5[(pd.notna(d5.age_at_diagnosis))&(d5.visit_month==0)].copy()
t4 = pd.merge(t3, d5[['participant_id', 'age_at_diagnosis']], on='participant_id',  how='left')

In [59]:
# MDS-UPDRS or originalUPDRS
## Most of LC and some PD recorded oridinal UPDRS rather than MDS

# MDS-UPDRS I
d6 = pd.read_csv('/data/CARD/PD/AMP-PD/Clinical/2021_v2_5release/clinical/MDS_UPDRS_Part_I.csv').drop_duplicates()
t5 = pd.merge(t4, d6[['participant_id',  'visit_name', 'visit_month', 'mds_updrs_part_i_summary_score']], 
              on=['participant_id', 'visit_name',  'visit_month'], how='left')

# MDS-UPDRS II
d7 = pd.read_csv('/data/CARD/PD/AMP-PD/Clinical/2021_v2_5release/clinical/MDS_UPDRS_Part_II.csv').drop_duplicates()
t6 = pd.merge(t5, d7[['participant_id',  'visit_name', 'visit_month', 'mds_updrs_part_ii_summary_score']], 
              on=['participant_id', 'visit_name',  'visit_month'], how='left')

# MDS-UPDRS III
d8 = pd.read_csv('/data/CARD/PD/AMP-PD/Clinical/2021_v2_5release/clinical/MDS_UPDRS_Part_III.csv').drop_duplicates()
t7 = pd.merge(t6, d8[['participant_id',  'visit_name', 'visit_month', 'mds_updrs_part_iii_summary_score', 'upd23b_clinical_state_on_medication']],  
              on=['participant_id', 'visit_name',  'visit_month'], how='left')

# MDS-UPDRS IV
d9 = pd.read_csv('/data/CARD/PD/AMP-PD/Clinical/2021_v2_5release/clinical/MDS_UPDRS_Part_IV.csv').drop_duplicates()
t8 = pd.merge(t7, d9[['participant_id', 'visit_name', 'visit_month', 'mds_updrs_part_iv_summary_score']],
              on=['participant_id', 'visit_name',  'visit_month'], how='left')

# original UPDRS part I/II/III/IV
d10 = pd.read_csv('/data/CARD/PD/AMP-PD/Clinical/2021_v2_5release/clinical/UPDRS.csv').drop_duplicates()
t9 = pd.merge(t8, d10[['participant_id', 'visit_name', 'visit_month', 'updrs1_ment_behav_mood_score','updrs2_adl_score',
                       'updrs3_motor_examination_score','updrs4_therapy_complications_score',
#                        'updrs1_on_medication', 'updrs2_on_medication'
                      ]],
              on=['participant_id', 'visit_name',  'visit_month'], how='left')

/usr/local/Anaconda/envs/py3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/Anaconda/envs/py3.8/lib/python3.8/site-packages/pandas/core/reshape/merge.py:1214: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(


In [67]:
d = t9.copy()
d['DATASET'] = d.participant_id.str.slice(0,2) # LD, PP, PD
d.pivot_table(index=['visit_month','visit_name'], columns='DATASET', values='participant_id', aggfunc='count')

DATASET                    LC      PD      PP
visit_month visit_name                       
-1          SC            8.0     NaN     1.0
 0          M0          533.0  1349.0  1827.0
            M0#2          1.0     NaN     NaN
 6          M6            NaN     NaN     3.0
 12         M12          80.0   556.0     5.0
 24         M24          96.0   489.0   129.0
            M24#2         2.0     NaN     NaN
            M24#3         1.0     NaN     NaN
 36         M36          95.0   391.0   118.0
            M36#2         6.0     NaN     NaN
 42         M42           NaN     NaN    17.0
 48         M48          90.0   121.0    44.0
 54         M54           NaN     NaN     4.0
 60         M60          37.0    38.0    12.0
            M60#2         1.0     NaN     NaN
 72         M72           8.0     NaN    61.0
 84         M84           NaN     NaN   151.0
 96         M96           NaN     NaN   159.0
 108        M108          NaN     NaN    29.0

In [68]:
# reduce to visit_name==0
d = d[d.visit_name.isin(['M0', 'M12','M24','M36', 'M48'])].copy()

In [73]:
d.shape

(5923, 23)

In [75]:
d.drop_duplicates(subset=['participant_id', 'visit_name']).shape

(5923, 23)

In [76]:
# Extract the META5 variants and make some tiny PLINK binaries 
snpmaps = {0:['rs2230288','T'],  # GBA E326K
        1:['rs76763715','C'],  # GBA N370N
        2:['rs34637584','A'],  # LRRK2 G2019S
        3:['rs104893877','T'],   # SNCA A53T
          }
pd.DataFrame.from_dict(snpmaps,orient='index',columns=['ID', 'ALT']).to_csv('snpslist.txt',header=None, index=False,sep='\t')

In [78]:
uid = d.participant_id.unique()
pd.DataFrame({'#FID':uid, 'IID':uid}).to_csv('idlist.txt', index=False, sep='\t')

In [79]:
plink_file_path = '/data/CARD/PD/AMP-PD/Plink/2021_v2_5release/pfile/amp_v2.5_allchr'

t = submitTerminal(plink2_cmd(f'{plink_file_path}', 'snps', f'--keep idlist.txt --extract snpslist.txt --export A --export-allele snpslist.txt'), message='extract snps of interest from the PLINK2 AMP v2.5 files')

EXEC_TIME in sec: 153.507 : extract snps of interest from the PLINK2 AMP v2.5 files 



In [80]:
# seems that all 5 variants successfully retrieved (sometimes, rsID are missing and can be missed by rsID based extraction)
t = pd.read_csv('snps.raw', sep='\t')
t = t.rename(columns=
    {'IID':'participant_id',
     'rs76763715_C':'GBA_N370N', 
     'rs2230288_T':'GBA_E326K',
     'rs104893877_T':'SNCA_A53T',
     'rs34637584_A':'LRRK2_G2019S'}
)
t = t.drop(columns=['FID', 'PAT', 'MAT', 'SEX','PHENOTYPE'])

In [82]:
# combine with clinical data
t2 = pd.merge(d, t, on='participant_id', how='left')
t3 = pd.read_csv('/data/CARD/PD/AMP-PD/Plink/2021_v2_5release/euro_king_pca_v2.5_July2021/genetic_ancestry_all_pca.csv')
t3 = t3.rename(columns={'IID':'participant_id'}).drop(columns=['#FID'])
df = pd.merge(t2, t3, on='participant_id', how='left')

In [83]:
df.shape

(5923, 38)

In [87]:
df.pivot_table(index=['visit_month','visit_name'], columns='DATASET', values='participant_id', aggfunc='count')

,DATASET,LC,PD,PP
visit_month,visit_name,,,
0,M0,533,1349,1827
12,M12,80,556,5
24,M24,96,489,129
36,M36,95,391,118
48,M48,90,121,44


In [89]:
df.loc[df.DATASET!='PP', ['participant_id', 'visit_name', 'visit_month', 'upsit_total_score',
    'age_at_baseline', 'sex', 'study_arm', 'prodromal_category',
       'diagnosis_at_baseline', 'diagnosis_latest',
       'case_control_other_at_baseline',
       'age_at_diagnosis', 'mds_updrs_part_i_summary_score',
       'mds_updrs_part_ii_summary_score', 'mds_updrs_part_iii_summary_score',
       'upd23b_clinical_state_on_medication',
       'mds_updrs_part_iv_summary_score', 'updrs1_ment_behav_mood_score',
       'updrs2_adl_score', 'updrs3_motor_examination_score',
       'updrs4_therapy_complications_score', 'DATASET', 'GBA_N370N',
       'GBA_E326K', 'SNCA_A53T', 'LRRK2_G2019S',
       'InfPop','PC1', 'PC2', 'PC3',
       'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10']].to_csv('UPSIT_genetics_LC_PD.csv', index=False)

In [45]:
df.columns

Index(['participant_id', 'visit_name', 'visit_month', 'upsit_total_score',
       'age_at_baseline', 'sex', 'study_arm', 'prodromal_category',
       'diagnosis_at_baseline', 'diagnosis_latest',
       'case_control_other_at_baseline', 'case_control_other_latest',
       'age_at_diagnosis', 'mds_updrs_part_i_summary_score',
       'mds_updrs_part_ii_summary_score', 'mds_updrs_part_iii_summary_score',
       'upd23b_clinical_state_on_medication',
       'mds_updrs_part_iv_summary_score', 'updrs1_ment_behav_mood_score',
       'updrs2_adl_score', 'updrs3_motor_examination_score',
       'updrs4_therapy_complications_score', 'DATASET', 'GBA_N370N',
       'GBA_E326K', 'SNCA_A53T', 'LRRK2_G2019S', 'InfPop', 'PC1', 'PC2', 'PC3',
       'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10'],
      dtype='object')

In [50]:
df.pivot_table(index=['DATASET', 'study_arm'], columns='visit_name', values='participant_id', aggfunc='count')

visit_name                M0
DATASET study_arm           
PD      Disease Control   60
        Healthy Control  480
        PD               809

# Get the genetic PCs for all AMP-PD samples + GUID

In [19]:
# Demographics, enrollment category case-control status
d2 = pd.read_csv('/data/CARD/PD/AMP-PD/Clinical/2021_v2_5release/clinical/Demographics.csv').drop_duplicates()
d3 = pd.read_csv('/data/CARD/PD/AMP-PD/Clinical/2021_v2_5release/clinical/Enrollment.csv').drop_duplicates()
d4 = pd.read_csv('/data/CARD/PD/AMP-PD/Clinical/2021_v2_5release/amp_pd_case_control.csv').drop_duplicates()
t2 = pd.merge(d2[['participant_id', 'GUID',  'age_at_baseline', 'sex']], d3[['participant_id', 'study_arm', 'prodromal_category']], on='participant_id', how='left')
t3 = pd.merge(t2, d4, on='participant_id',  how='left').drop_duplicates()

In [20]:
# age at diagnosis (at visit_month==0)
d5 = pd.read_csv('/data/CARD/PD/AMP-PD/Clinical/2021_v2_5release/clinical/PD_Medical_History.csv').drop_duplicates()
d5 = d5[(pd.notna(d5.age_at_diagnosis))&(d5.visit_month==0)].copy()
t4 = pd.merge(t3, d5[['participant_id', 'age_at_diagnosis']], on='participant_id',  how='left')

In [21]:
# genetic PCs
t3 = pd.read_csv('/data/CARD/PD/AMP-PD/Plink/2021_v2_5release/euro_king_pca_v2.5_July2021/genetic_ancestry_all_pca.csv')
t3 = t3.rename(columns={'IID':'participant_id'}).drop(columns=['#FID'])
df = pd.merge(t4, t3, on='participant_id', how='right')

In [22]:
# add dataset
df['DATASET'] = df.participant_id.str.slice(0,2) # LD, PP, PD

In [23]:
df.to_csv("demographicsPCs.csv", index=False)

In [26]:
df.columns

Index(['participant_id', 'GUID', 'age_at_baseline', 'sex', 'study_arm',
       'prodromal_category', 'diagnosis_at_baseline', 'diagnosis_latest',
       'case_control_other_at_baseline', 'case_control_other_latest',
       'age_at_diagnosis', 'InfPop', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6',
       'PC7', 'PC8', 'PC9', 'PC10', 'DATASET'],
      dtype='object')